In [ ]:
!pip install torch torchvision
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
# import torchvision.transforms as transforms
from torchvision import datasets, transforms
import torch.optim as optim

** Use Group or Depthwise Convolutions**

  This strategy drastically reduces parameters by splitting convolutions across channel groups instead of applying full convolutions over all channels.
  It’s a core idea used in MobileNet and other efficient CNN architectures.

What changes

In your model, the ConvBlock4 layer is the biggest contributor to parameters.
We’ll replace it with a depthwise separable convolution:

 * Depthwise convolution — applies a separate kernel per input channel (parameter count much smaller).

 * Pointwise convolution — 1×1 convolution to combine features (cheap relative to full conv).


This reduces parameters massively while preserving receptive field and performance.

---

### **CNN Parameter Reduction Strategies — Cheat Sheet**

#### **1. Reduce Output Channels**

* **How:** Lower the number of output channels in heavy convolution layers.
* **When:** When a layer has disproportionately large parameters (usually deeper layers).
* **Formula:**
  [
  \text{Params} = C_{\text{in}} × C_{\text{out}} × K × K
  ]
* **Example:** Changing Conv2d(128 → 215, 3×3) → Conv2d(128 → 144, 3×3) drastically reduces parameters.

---

#### **2. 1×1 Convolution (Bottleneck Layer)**

* **How:** Insert a 1×1 convolution before expensive convolution layers to reduce channel depth.
* **When:** Before layers where (C_{\text{in}}) is high, to compress features before heavy computation.
* **Formula:**
  [
  \text{Params}*{\text{bottleneck}} = C*{\text{in}} × C_{\text{reduced}} × 1 × 1
  ]
  [
  \text{Params}*{\text{conv}} = C*{\text{reduced}} × C_{\text{out}} × K × K
  ]
* **Example:**
  128 → 96 channels bottleneck before ConvBlock4 can cut parameters significantly.

---

#### **3. Depthwise Separable Convolution**

* **How:** Replace a normal convolution with depthwise convolution followed by a pointwise convolution.
* **When:** When aiming for large parameter savings without losing much performance (mobile-friendly design).
* **Formula:**
  [
  \text{Params} = C_{\text{in}} × K × K + C_{\text{in}} × C_{\text{out}} × 1 × 1
  ]
* **Example:**
  ConvBlock4: 128 → 215 channels (3×3)
  Normal: (128×215×3×3 = 247,680)
  Depthwise separable: (128×3×3 + 128×215×1×1 = 28,672) (~8× reduction).

---

#### **General Variables**

* (C_in) = Input channels
* (C_out) = Output channels
* (K) = Kernel size

---

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ciferNet(nn.Module):
    def __init__(self):
        super(ciferNet, self).__init__()

        # Conv Block 1
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=2, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
        )

        # Conv Block 2
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=2, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(64),
        )

        # Conv Block 3
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(128),
        )

        # Conv Block 4 → Depthwise separable convolution
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1, groups=128, bias=False),  # Depthwise conv
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(1, 1), bias=False),  # Pointwise conv
            nn.ReLU(),
            nn.BatchNorm2d(256),
        )

        # GAP and classifier
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Conv2d(in_channels=256, out_channels=10, kernel_size=(1, 1), bias=False)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.gap(x)
        x = self.classifier(x)
        x = x.view(x.size(0), -1)
        return F.log_softmax(x, dim=-1)


In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = ciferNet().to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
              ReLU-2           [-1, 16, 32, 32]               0
       BatchNorm2d-3           [-1, 16, 32, 32]              32
            Conv2d-4           [-1, 32, 16, 16]           4,608
              ReLU-5           [-1, 32, 16, 16]               0
       BatchNorm2d-6           [-1, 32, 16, 16]              64
            Conv2d-7           [-1, 64, 16, 16]          18,432
              ReLU-8           [-1, 64, 16, 16]               0
       BatchNorm2d-9           [-1, 64, 16, 16]             128
           Conv2d-10             [-1, 64, 8, 8]          36,864
             ReLU-11             [-1, 64, 8, 8]               0
      BatchNorm2d-12             [-1, 64, 8, 8]             128
           Conv2d-13            [-1, 128, 8, 8]          73,728
             ReLU-14            [-

In [ ]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.4741, 0.4727, 0.4733),(0.2521, 0.2520, 0.2506),)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.4741, 0.4727, 0.4733),(0.2521, 0.2520, 0.2506),)
                                       ])

In [ ]:
train = datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
test = datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)

In [ ]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
train_dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)
test_dataloader_args = dict(shuffle=False, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=False, batch_size=64)


# train dataloader
train_loader = torch.utils.data.DataLoader(train, **train_dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **test_dataloader_args)

CUDA Available? True


In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss.item()) # Added .item()

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.float().argmax(dim=1, keepdim=True)  # get the index of the max log-probability, convert to float
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.float().argmax(dim=1, keepdim=True)  # get the index of the max log-probability, convert to float
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss) # Added .item()

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [ ]:
from torch.optim.lr_scheduler import StepLR

model =  ciferNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# scheduler = StepLR(optimizer, step_size=6, gamma=0.1)


EPOCHS = 8
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    # scheduler.step()
    test(model, device, test_loader)

EPOCH: 0


Loss=1.31123948097229 Batch_id=390 Accuracy=46.62: 100%|██████████| 391/391 [00:19<00:00, 20.07it/s]



Test set: Average loss: 1.2479, Accuracy: 5539/10000 (55.39%)

EPOCH: 1


Loss=1.0990206003189087 Batch_id=390 Accuracy=62.07: 100%|██████████| 391/391 [00:14<00:00, 27.61it/s]



Test set: Average loss: 1.0047, Accuracy: 6349/10000 (63.49%)

EPOCH: 2


...


Test set: Average loss: 0.7135, Accuracy: 7507/10000 (75.07%)

EPOCH: 6


Loss=0.4977995455265045 Batch_id=390 Accuracy=83.12: 100%|██████████| 391/391 [00:14<00:00, 27.29it/s]



Test set: Average loss: 0.6444, Accuracy: 7776/10000 (77.76%)

EPOCH: 7


Loss=0.4013732969760895 Batch_id=390 Accuracy=85.24: 100%|██████████| 391/391 [00:13<00:00, 28.18it/s]



Test set: Average loss: 0.6742, Accuracy: 7720/10000 (77.20%)

